In [ ]:
import xarray as xr
import fsspec
from distributed import Client
import hvplot.xarray
import numpy as np
import hvplot.xarray  # noqa: adds hvplot method to xarray objects
import holoviews as hv
import geoviews as gv
from geoviews import opts
import pandas as pd
import panel as pn
import holoviews as hv
from datetime import datetime, timedelta
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import s3fs
import warnings
warnings.filterwarnings('ignore')

from IPython.core.display import display, HTML
display(HTML("""<style>
@media (min-width: 1200px) {
  .container {
    width: 98%;
  }
}

div.output_subarea{padding:.4em .4em 0 .4em;-webkit-box-flex:1;-moz-box-flex:1;box-flex:1;flex:1;max-width:100%}
.bk-input {background-color:blue}
</style>
<script>
    $('#appmode-leave').hide();                          // Hides the edit app button.
    $('#appmode-busy').hide();                           // Hides the kernel busy indicator.
    $('#appmode-loader').append('<h2>Loading...</h2>');  // Adds a loading message.
;</script>

"""))


In [ ]:
client = Client()

In [ ]:
latlngbox = [-82, -74, 44.5, 49]
climatic_period = ['1981-01-01', '2010-12-31']

In [ ]:
bucket = 's3://cires-20-century-reanalysis-v3/zarr/single-levels-space'
storage_options = {'endpoint_url': 'https://s3.us-east-1.wasabisys.com'}

In [ ]:
ds = xr.open_zarr(fsspec.get_mapper(bucket,
                                    client_kwargs=storage_options,
                                    anon=True),
                  consolidated=True)
# ds = xr.open_zarr('/home/slanglois/Documents/store/final/target.zarr',
#                   consolidated=True)


In [ ]:
end_date = '1979-09-25'
start_date = (datetime.strptime(end_date, '%Y-%m-%d') - timedelta(days=2)).strftime('%Y-%m-%d')


ds_ls_sliced = ds.sel(time=slice(start_date, end_date)).load()


ds_sliced = ds_ls_sliced.sel(longitude=slice(latlngbox[0], latlngbox[1]),
                             latitude=slice(latlngbox[2], latlngbox[3]))

In [ ]:
da_values = ds_sliced['prate'].max('time')
df_da = da_values.to_dataframe().reset_index()
gdf = gpd.GeoDataFrame(
    df_da, geometry=gpd.points_from_xy(df_da.longitude, df_da.latitude))
gdf.crs = 4326
gdf = gdf.to_crs(3857)
gdf['longitude'] = gdf.geometry.x
gdf['latitude'] = gdf.geometry.y


In [ ]:
# ruled-based algorithm

def preliminary_type(ds_rolling_sl,
                     ds_rolling_pl):


    # 1. 0.12 < Precipitation rate <= 0.5 and CAPE < 500
    a = xr.ufuncs.logical_and(ds_rolling_sl.prate <= 0.5/3600, ds_rolling_sl.cape<500)
    b = xr.ufuncs.logical_and(ds_rolling_sl.prate >0.12/3600, a)
    c = xr.ufuncs.logical_and(a,b)
    da = xr.where(c, 40, -1)

    # 2. 0.12 < Precipitation rate <= 0.5 and CAPE >= 500
    a = xr.ufuncs.logical_and(ds_rolling_sl.prate <= 0.5/3600, ds_rolling_sl.cape>=500)
    b = xr.ufuncs.logical_and(ds_rolling_sl.prate >0.12/3600, a)
    c = xr.ufuncs.logical_and(a,b)
    da = xr.where(c, 40, da)

    # 3. Precipitation rate > 0.5, CAPE < 500, 500mb Pressure gradient <=9.5, 900 mb Pressure gradient < 8
    a = xr.ufuncs.logical_and(ds_rolling_sl.prate>0.5/3600 ,ds_rolling_sl.cape<500)
    b = xr.ufuncs.logical_and(ds_rolling_pl.gradh500<=9.5 ,ds_rolling_pl.gradh900<8)
    c = xr.ufuncs.logical_and(a,b)
    da = xr.where(c, 30, da)

    # 4. Precipitation rate > 0.5, CAPE >= 500, 500mb Pressure gradient <=9.5, 900 mb Pressure gradient < 8
    a = xr.ufuncs.logical_and(ds_rolling_sl.prate>0.5/3600 ,ds_rolling_sl.cape>=500)
    b = xr.ufuncs.logical_and(ds_rolling_pl.gradh500<=9.5 ,ds_rolling_pl.gradh900<8)
    c = xr.ufuncs.logical_and(a,b)
    da = xr.where(c, 33, da)

    # 5. Precipitation rate > 0.5, CAPE < 500, 500mb Pressure gradient >9.5, 900 mb Pressure gradient >= 8
    a = xr.ufuncs.logical_and(ds_rolling_sl.prate>0.5/3600 ,ds_rolling_sl.cape<500)
    b = xr.ufuncs.logical_and(ds_rolling_pl.gradh500>9.5 ,ds_rolling_pl.gradh900>=8)
    c = xr.ufuncs.logical_and(a,b)
    da = xr.where(c, 10, da)

    # 6. Precipitation rate > 0.5, CAPE >= 500, 500mb Pressure gradient >9.5, 900 mb Pressure gradient >= 8
    a = xr.ufuncs.logical_and(ds_rolling_sl.prate>0.5/3600 ,ds_rolling_sl.cape>=500)
    b = xr.ufuncs.logical_and(ds_rolling_pl.gradh500>9.5 ,ds_rolling_pl.gradh900>=8)
    c = xr.ufuncs.logical_and(a,b)
    da = xr.where(c, 13, da)

    # 7. Precipitation rate > 0.5, CAPE < 500, 500mb Pressure gradient <=9.5, 900 mb Pressure gradient >= 8
    a = xr.ufuncs.logical_and(ds_rolling_sl.prate>0.5/3600 ,ds_rolling_sl.cape<500)
    b = xr.ufuncs.logical_and(ds_rolling_pl.gradh500<=9.5 ,ds_rolling_pl.gradh900>=8)
    c = xr.ufuncs.logical_and(a,b)
    da = xr.where(c, 60, da)

    # 8. Precipitation rate > 0.5, CAPE >= 500, 500mb Pressure gradient <=9.5, 900 mb Pressure gradient >= 8
    a = xr.ufuncs.logical_and(ds_rolling_sl.prate>0.5/3600 ,ds_rolling_sl.cape>=500)
    b = xr.ufuncs.logical_and(ds_rolling_pl.gradh500<=9.5 ,ds_rolling_pl.gradh900>=8)
    c = xr.ufuncs.logical_and(a,b)
    da = xr.where(c, 63, da)

    # 9. Precipitation rate > 0.5, CAPE < 500, 500mb Pressure gradient >9.5, 900 mb Pressure gradient < 8
    a = xr.ufuncs.logical_and(ds_rolling_sl.prate>0.5/3600 ,ds_rolling_sl.cape<500)
    b = xr.ufuncs.logical_and(ds_rolling_pl.gradh500>9.5 ,ds_rolling_pl.gradh900<8)
    c = xr.ufuncs.logical_and(a,b)
    da = xr.where(c, 60, da)

    # 10. Precipitation rate > 0.5, CAPE >= 500, 500mb Pressure gradient >9.5, 900 mb Pressure gradient < 8
    a = xr.ufuncs.logical_and(ds_rolling_sl.prate>0.5/3600 ,ds_rolling_sl.cape>=500)
    b = xr.ufuncs.logical_and(ds_rolling_pl.gradh500>9.5 ,ds_rolling_pl.gradh900<8)
    c = xr.ufuncs.logical_and(a,b)
    da = xr.where(c, 63, da)

    # 10. Precipitation rate <0.12
    da = xr.where(ds_rolling_sl.prate<=0.12/3600, 99, da)
    return pd.DataFrame(da.values[::-1,:])

def preliminary_type_da(ds_rolling_sl,
                        ds_rolling_pl):


    # 1. 0.12 < Precipitation rate <= 0.5 and CAPE < 500
    a = xr.ufuncs.logical_and(ds_rolling_sl.prate <= 0.5/3600, ds_rolling_sl.cape<500)
    b = xr.ufuncs.logical_and(ds_rolling_sl.prate >0.12/3600, a)
    c = xr.ufuncs.logical_and(a,b)
    da = xr.where(c, 40, -1)

    # 2. 0.12 < Precipitation rate <= 0.5 and CAPE >= 500
    a = xr.ufuncs.logical_and(ds_rolling_sl.prate <= 0.5/3600, ds_rolling_sl.cape>=500)
    b = xr.ufuncs.logical_and(ds_rolling_sl.prate >0.12/3600, a)
    c = xr.ufuncs.logical_and(a,b)
    da = xr.where(c, 40, da)

    # 3. Precipitation rate > 0.5, CAPE < 500, 500mb Pressure gradient <=9.5, 900 mb Pressure gradient < 8
    a = xr.ufuncs.logical_and(ds_rolling_sl.prate>0.5/3600 ,ds_rolling_sl.cape<500)
    b = xr.ufuncs.logical_and(ds_rolling_pl.gradh500<=9.5 ,ds_rolling_pl.gradh900<8)
    c = xr.ufuncs.logical_and(a,b)
    da = xr.where(c, 30, da)

    # 4. Precipitation rate > 0.5, CAPE >= 500, 500mb Pressure gradient <=9.5, 900 mb Pressure gradient < 8
    a = xr.ufuncs.logical_and(ds_rolling_sl.prate>0.5/3600 ,ds_rolling_sl.cape>=500)
    b = xr.ufuncs.logical_and(ds_rolling_pl.gradh500<=9.5 ,ds_rolling_pl.gradh900<8)
    c = xr.ufuncs.logical_and(a,b)
    da = xr.where(c, 33, da)

    # 5. Precipitation rate > 0.5, CAPE < 500, 500mb Pressure gradient >9.5, 900 mb Pressure gradient >= 8
    a = xr.ufuncs.logical_and(ds_rolling_sl.prate>0.5/3600 ,ds_rolling_sl.cape<500)
    b = xr.ufuncs.logical_and(ds_rolling_pl.gradh500>9.5 ,ds_rolling_pl.gradh900>=8)
    c = xr.ufuncs.logical_and(a,b)
    da = xr.where(c, 10, da)

    # 6. Precipitation rate > 0.5, CAPE >= 500, 500mb Pressure gradient >9.5, 900 mb Pressure gradient >= 8
    a = xr.ufuncs.logical_and(ds_rolling_sl.prate>0.5/3600 ,ds_rolling_sl.cape>=500)
    b = xr.ufuncs.logical_and(ds_rolling_pl.gradh500>9.5 ,ds_rolling_pl.gradh900>=8)
    c = xr.ufuncs.logical_and(a,b)
    da = xr.where(c, 13, da)

    # 7. Precipitation rate > 0.5, CAPE < 500, 500mb Pressure gradient <=9.5, 900 mb Pressure gradient >= 8
    a = xr.ufuncs.logical_and(ds_rolling_sl.prate>0.5/3600 ,ds_rolling_sl.cape<500)
    b = xr.ufuncs.logical_and(ds_rolling_pl.gradh500<=9.5 ,ds_rolling_pl.gradh900>=8)
    c = xr.ufuncs.logical_and(a,b)
    da = xr.where(c, 60, da)

    # 8. Precipitation rate > 0.5, CAPE >= 500, 500mb Pressure gradient <=9.5, 900 mb Pressure gradient >= 8
    a = xr.ufuncs.logical_and(ds_rolling_sl.prate>0.5/3600 ,ds_rolling_sl.cape>=500)
    b = xr.ufuncs.logical_and(ds_rolling_pl.gradh500<=9.5 ,ds_rolling_pl.gradh900>=8)
    c = xr.ufuncs.logical_and(a,b)
    da = xr.where(c, 63, da)

    # 9. Precipitation rate > 0.5, CAPE < 500, 500mb Pressure gradient >9.5, 900 mb Pressure gradient < 8
    a = xr.ufuncs.logical_and(ds_rolling_sl.prate>0.5/3600 ,ds_rolling_sl.cape<500)
    b = xr.ufuncs.logical_and(ds_rolling_pl.gradh500>9.5 ,ds_rolling_pl.gradh900<8)
    c = xr.ufuncs.logical_and(a,b)
    da = xr.where(c, 60, da)

    # 10. Precipitation rate > 0.5, CAPE >= 500, 500mb Pressure gradient >9.5, 900 mb Pressure gradient < 8
    a = xr.ufuncs.logical_and(ds_rolling_sl.prate>0.5/3600 ,ds_rolling_sl.cape>=500)
    b = xr.ufuncs.logical_and(ds_rolling_pl.gradh500>9.5 ,ds_rolling_pl.gradh900<8)
    c = xr.ufuncs.logical_and(a,b)
    da = xr.where(c, 63, da)

    # 10. Precipitation rate <0.12
    da = xr.where(ds_rolling_sl.prate<=0.12/3600, 99, da)
    return da

In [ ]:
# Find important storms


# Compute preliminary storm type and add to cloud

In [ ]:
bucket = 's3://cires-20-century-reanalysis-v3/zarr/pressure-levels-space'

storage_options = {'endpoint_url': 'https://s3.us-east-1.wasabisys.com'}

# ds_pl = xr.open_zarr('/home/slanglois/Documents/store/pressure/final/target.zarr', consolidated=True)

ds_pl = xr.open_zarr(fsspec.get_mapper(bucket,
                                    client_kwargs=storage_options,
                                    anon=True),
                     consolidated=True)


In [ ]:
ds_ls_pl_sliced = ds_pl.sel(level=[500,900], time=slice(start_date, end_date)).load()

level = 500
variable = 'gradh500'

da = np.sqrt(np.square(ds_ls_pl_sliced.sel(level=level).hgt.differentiate('longitude')) + 
             np.square(ds_ls_pl_sliced.sel(level=level).hgt.differentiate('latitude')))
ds_ls_pl_sliced[variable] = da

level = 900
variable = 'gradh900'

da = np.sqrt(np.square(ds_ls_pl_sliced.sel(level=level).hgt.differentiate('longitude')) + 
             np.square(ds_ls_pl_sliced.sel(level=level).hgt.differentiate('latitude')))
ds_ls_pl_sliced[variable] = da

ds_pl_sliced = ds_ls_pl_sliced.sel(longitude=slice(latlngbox[0], latlngbox[1]),
                             latitude=slice(latlngbox[2], latlngbox[3]))

In [ ]:
df = preliminary_type(ds_sliced.max('time'),
                      ds_pl_sliced.max('time'))

ds_sliced['prelim_type'] = preliminary_type_da(ds_sliced.max('time'),
                                               ds_pl_sliced.max('time'))

In [ ]:
numerical_code = [10, 13, 30, 33, 40, 43, 60, 63, 99]
gspec = pn.GridSpec(sizing_mode='stretch_both', height=100)


fs = s3fs.S3FileSystem(anon=True,
                       client_kwargs= {'endpoint_url': 'https://s3.us-east-2.wasabisys.com'})

for x in range(0,5):
    for y in range(0,9):
        gspec[x, y] = pn.widgets.Select(value=10, options=numerical_code, max_height=25, max_width=50,
                                        margin=(0,0,0,0), background='black')
        
# look for a storm type calculated by meteorologist
filename = 's3://analytics-store/manual_storm_types/{}.csv'.format(end_date)
if fs.exists(filename):
    with fs.open(filename) as f:
        df = pd.read_csv(f)
        for i in range(0,5):
            for j in range(0,9):
                gspec[i, j].value = df.iloc[i,j]

# if empty, plot preliminary storm type
else:
    df = preliminary_type(ds_sliced.max('time'),
                          ds_pl_sliced.max('time'))
    for i in range(0,5):
        for j in range(0,9):
            gspec[i, j].value = df.iloc[i,j]   

def plot_variable_ls(ds, ds_ls, variable, gdf, title, numbers, shared):
    da_values = ds[variable]
    
        
    if variable == 'prelim_type':
        da_values_ls = ds_ls['tcdc']
    else:
        da_values_ls = ds_ls[variable]
        
    precise = False
    
    if variable == 'prate':
        da_values = da_values*3600
        da_values_ls = da_values_ls*3600
        precise = True
    elif variable =='tmax' or variable=='tmin':
        da_values = da_values - 273.15
        da_values_ls = da_values_ls - 273.15
    

    # 
    df_da = da_values.to_dataframe().reset_index()
    
    if precise==True:
        labels = hv.Labels({('x', 'y'): gdf[['longitude','latitude']],
                            'text': ['{0:.1f}'.format(i) for i in df_da[variable]]},
                                                   ['x', 'y'], 'text') 
    else:
        labels = hv.Labels({('x', 'y'): gdf[['longitude','latitude']],
                            'text': ['{0:.0f}'.format(i) for i in df_da[variable]]},
                                                   ['x', 'y'], 'text') 
    clim = (da_values_ls.min().values, da_values_ls.max().values)
    # 
    jet = cm.get_cmap('jet', 256)
    newcolors = jet(np.linspace(0, 1, 256))
    newcolors[75, :] = [0.9,0.9,0.9,0]
#     newcolors[0, :] = [0.9,0.9,0.9,0]
    cmap = ListedColormap(newcolors[75:225, :])
#     cmap = ListedColormap(newcolors)
    
    grid_plot = da_values.hvplot(x='longitude', y='latitude', 
                                 cmap=cmap,
                                 title=title,
                                 clim=clim,
                                 alpha=0.9,
                                 geo=True)

    shared_axes = shared.value
    display_number = numbers.value
    
    xlim=(-84, -72)
    ylim=(44,50)
    
    if variable == 'hgt':
        display_number = False
        xlim=(-98, -50)
        ylim=(39,64)
    
    alpha = 0.65
    if variable == 'prelim_type':
        alpha = 0
    contourf_plot = da_values_ls.hvplot.contourf(levels=10,
                                                 geo=True,
                                                 hover=True,
                                                 clim=clim,
                                                 alpha=alpha,
                                                 cmap=cmap,
                                                 xlim=xlim,
                                                 ylim=ylim,
                                                 width=525,
                                                 height=250,
                                                 tiles='ESRI')
    # 
#     contour_plot = da_values_ls.hvplot.contour(levels=20,
#                                                geo=True,
#                                                clim=clim,
#                                                xlim=(-84, -72),
#                                                ylim=(44,50),
#                                                alpha=1,
#                                                cmap='isolum')
    
    # 

    

    
    if display_number is True:
        overlay =  (contourf_plot*grid_plot*labels).opts(active_tools=['wheel_zoom','pan'],
                                                         toolbar = None,
                                                         shared_axes=shared_axes)
    else:
        overlay =  (contourf_plot*grid_plot).opts(active_tools=['wheel_zoom','pan'],
                                                  toolbar = None,
                                                  shared_axes=shared_axes)
    
        
    overlay.opts(
        opts.Labels(text_font_size='8.5pt', text_font_style='bold'))
    return overlay

# Replace by important storms
options=['Temps sec : 1979-09-23 - 1979-09-25',
                               'Tempête #1 : 1979-09-18 - 1979-09-20']

dates = '1979-09-23 - 1979-09-25'

x = pn.widgets.Select(value=options[0], 
                      options=options, 
                      name='# de tempête')
progress = pn.widgets.Progress(name='Progress', value=0, width=200)

numbers = pn.widgets.Checkbox(name='Display numbers', value=True)
shared = pn.widgets.Checkbox(name='Shared axis', value=False)

layout = pn.Column(
            pn.Row('## Weather Vizualisation (72h window)', pn.Column(x, progress, pn.Row(numbers, shared)),
                   pn.Column(gspec, sizing_mode='stretch_width')),
                pn.Row(
                plot_variable_ls(ds_sliced.max('time'), ds_ls_sliced.max('time'), 'cape', gdf, 
                                 '{} : maximum CAPE (J/kg)'.format(dates), numbers, shared),
                plot_variable_ls(ds_sliced.max('time'), ds_ls_sliced.max('time'), 'prate', gdf,
                                 '{} : maximum precipitation rate (mm/h)'.format(dates), numbers, shared),
                plot_variable_ls(ds_sliced.max('time'), ds_ls_sliced.max('time'), 'pr_wtr', gdf,
                                 '{} : maximum precipitable water (mm)'.format(dates), numbers, shared)
                ),
            pn.Row(
                plot_variable_ls(ds_sliced.max('time'), ds_ls_sliced.max('time'), 'tcdc', gdf,
                                 '1979-09-13 - 1979-09-15 : maximum cloud cover (%)', numbers, shared),
                plot_variable_ls(ds_pl_sliced.max('time'), ds_ls_pl_sliced.max('time'), 'gradh500', gdf,
                                 '1979-09-13 - 1979-09-15 : maximum 500-mb gradients (m/°)', numbers, shared)
                ,
                plot_variable_ls(ds_pl_sliced.max('time'), ds_ls_pl_sliced.max('time'), 'gradh900', gdf,
                                 '1979-09-13 - 1979-09-15 : maximum 900-mb gradients (m/°)', numbers, shared)
                ),
                pn.Row(
                plot_variable_ls(ds_sliced, ds_ls_sliced.max('time'), 'prelim_type', gdf,
                                 '1979-09-13 - 1979-09-15 : preliminary storm type', numbers, shared),
                plot_variable_ls(ds_pl_sliced.max('time').sel(level=500), ds_ls_pl_sliced.max('time').sel(level=500), 'hgt', gdf,
                                 '1979-09-13 - 1979-09-15 : maximum 500-mb height (m)', numbers, shared)
                ,
                plot_variable_ls(ds_pl_sliced.max('time').sel(level=900), ds_ls_pl_sliced.max('time').sel(level=900), 'hgt', gdf,
                                 '1979-09-13 - 1979-09-15 : maximum 900-mb height (m)', numbers, shared)
                )
            
            )





def update(event):
    end_date = x.value.split(' - ')[-1]
    start_date = (datetime.strptime(end_date, '%Y-%m-%d') - timedelta(days=2)).strftime('%Y-%m-%d')
    progress.value = 5
    ds_ls_sliced = ds.sel(time=slice(start_date,end_date)).load()
    progress.value = 15
    ds_ls_pl_sliced = ds_pl.sel(level=[500,900], time=slice(start_date,end_date)).load()
    progress.value = 25
    ###
    ds_sliced = ds_ls_sliced.sel(longitude=slice(latlngbox[0], latlngbox[1]),
                                 latitude=slice(latlngbox[2], latlngbox[3]))
    level = 500
    variable = 'gradh500'
    da = np.sqrt(np.square(ds_ls_pl_sliced.sel(level=level).hgt.differentiate('longitude')) + 
                 np.square(ds_ls_pl_sliced.sel(level=level).hgt.differentiate('latitude')))
    ds_ls_pl_sliced[variable] = da

    level = 900
    variable = 'gradh900'
    da = np.sqrt(np.square(ds_ls_pl_sliced.sel(level=level).hgt.differentiate('longitude')) + 
                 np.square(ds_ls_pl_sliced.sel(level=level).hgt.differentiate('latitude')))
    ds_ls_pl_sliced[variable] = da
    
    ds_pl_sliced = ds_ls_pl_sliced.sel(longitude=slice(latlngbox[0], latlngbox[1]),
                                 latitude=slice(latlngbox[2], latlngbox[3]))
    
    ds_sliced['prelim_type'] = preliminary_type_da(ds_sliced.max('time'),
                                               ds_pl_sliced.max('time'))
    
    variables = {'cape': ['cape', ds_sliced.max('time'), ds_ls_sliced.max('time'), 'maximum CAPE (J/kg)'],
                 'prate': ['prate', ds_sliced.max('time'), ds_ls_sliced.max('time'),'maximum precipitation rate (mm/h)'],
                 'pr_wtr' : ['pr_wtr', ds_sliced.max('time'), ds_ls_sliced.max('time'),'maximum precipitable water (mm)'],
                 'tcdc': ['tcdc', ds_sliced.max('time'), ds_ls_sliced.max('time'), 'maximum cloud cover (%)'],
                 'gradh500': ['gradh500', ds_pl_sliced.max('time'), ds_ls_pl_sliced.max('time'), 'maximum 500-mb gradients (m/°)'],
                 'gradh900': ['gradh900', ds_pl_sliced.max('time'), ds_ls_pl_sliced.max('time'), 'maximum 900-mb gradients (m/°)'],
                 'prelim_type': ['prelim_type', ds_sliced, ds_ls_sliced.max('time'), 'preliminary storm type'],
                 'hgt': ['hgt', ds_pl_sliced.max('time').sel(level=500), ds_ls_pl_sliced.max('time').sel(level=500), 'maximum 500-mb height (m)'],
                 'hgt2': ['hgt', ds_pl_sliced.max('time').sel(level=900), ds_ls_pl_sliced.max('time').sel(level=900), 'maximum 900-mb height (m)']}
    
    
    for i, (key, value) in enumerate(variables.items()):  
        layout[int(np.ceil((i+1)/3))][i - int(np.floor(i/3))*3].object = plot_variable_ls(value[1], value[2],
                                                 value[0], gdf, '{} - {} : {}'.format(start_date, end_date, value[3]), numbers, shared )
        if progress.value<=95:
            progress.value = progress.value + 5

    # look for a storm type calculated by meteorologist
    filename = 's3://analytics-store/manual_storm_types/{}.csv'.format(end_date)
    if fs.exists(filename):
        with fs.open(filename) as f:
            df = pd.read_csv(f)
            for i in range(0,5):
                if progress.value<=94:
                    progress.value = progress.value + 6
                for j in range(0,9):
                    gspec[i, j].value = df.iloc[i,j]
                    
                
    # if empty, plot preliminary storm type
    else:
        df = preliminary_type(ds_sliced.max('time'),
                              ds_pl_sliced.max('time'))
        for i in range(0,5):
            if progress.value<=94:
                progress.value = progress.value + 6
            for j in range(0,9):
                gspec[i, j].value = df.iloc[i,j] 
                
    progress.value = 0
    
    # TODO : Calculate preliminary storm type and replace code above



def update2(event):
    end_date = layout[0][1][0].value.split(' - ')[-1]
    arr = np.zeros([6,10])
    for i in range(0,5):
        for j in range(0,9):
            arr[i,j] = gspec[i, j].value
    with fs.open('s3://analytics-store/manual_storm_types/{}.csv'.format(end_date), 'w') as f:
        pd.DataFrame(arr).to_csv(f, index=False)

# def update2(event):
#     print('test')

x.param.watch(update, 'value')
numbers.param.watch(update, 'value')
shared.param.watch(update, 'value')

for i in range(0,5):
    for j in range(0,9):
        gspec[i, j].param.watch(update2, 'value')

layout